In [33]:
from gensim.models import Word2Vec
import numpy as np
from tensorflow import keras
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# from collections import Counter
# from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import RidgeClassifier
# from sklearn.metrics import f1_score
# from sklearn.svm import SVC
# from sklearn.ensemble import RandomForestClassifier

In [2]:
#数据处理
#读数据
df = pd.read_csv("data//train_set.csv",sep = '\t',nrows=20000)#index_col=0去掉索引
df.head()

,label,text
0,2,2967 6758 339 2021 1854 3731 4109 3792 4149 15...
1,11,4464 486 6352 5619 2465 4802 1452 3137 5778 54...
2,3,7346 4068 5074 3747 5681 6093 1777 2226 7354 6...
3,2,7159 948 4866 2109 5520 2490 211 3956 5520 549...
4,3,3646 3055 3055 2490 4659 6065 3370 5814 2465 5...


In [3]:
#数据复制
def copyNews(dataDf,tag):
    smallNewsDf = dataDf[dataDf['label']==tag]
    dataDf = pd.concat([dataDf,smallNewsDf,smallNewsDf])
    return dataDf
df = copyNews(df,13)
df = copyNews(df,12)
df = copyNews(df,11)
df = copyNews(df,10)

In [4]:
#3750，648，900排名前三的很可能是标点符号移除字符
def delWord(text):
    text = text.replace('3750','')
    text = text.replace('648','')
    text = text.replace('900','')
    return text
df['text'] =df['text'].apply(delWord)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22184 entries, 0 to 19996
Data columns (total 2 columns):
label    22184 non-null int64
text     22184 non-null object
dtypes: int64(1), object(1)
memory usage: 519.9+ KB


In [28]:
#word2vec训练词向量
#训练词向量、保存
def trainWord2vec(sents,savePath = 'word2vecModel'):
    model = Word2Vec(sents,
                     size=50,#词向量维度
                     window=12,#扫描句子窗口大小，即中心词与周围词最大距离，默认值为5#根据句子长短要改，注意
                     min_count = 2,#过滤低频词，默认5
                     max_vocab_size = None,#设置RAM限制
                     sample = 0.001,#默认0.001，高频词汇随机降采样阈值，范围（0,le-5），默认le-3
                     seed=1,
                     workers = 3,#训练模型的线程
                     min_alpha = 0.0001,#学习率，默认0.0001
                     sg = 1,#0为CBOW算法 周围词预测中心词，1为skip-gram算法 中心词预测周围词
                     hs = 0,#0 为negative sampling负采样，1为 hierarchical softmax 层级softmax 二叉树
                     negative = 5, #负采样设置多少个noise words噪声词，>0时会采用负采样。默认5
                     cbow_mean = 1,#只有使用cbow时起作用，即sg=0时。为1取周围词均值。为0时取周围词（上下文词）之和。
                     iter =3,#迭代遍历语料库的次数，默认5
                     null_word = 0,
                     trim_rule = None,#设置词汇表的整理规则，哪些词去留，None时min_count会被使用
                     sorted_vocab = 1,#1为降序，对word index词频排序，由多到少，默认1
                     batch_words = 10000,#默认10000，每批传词数量
                     compute_loss = False#是否计算loss，默认False
                    )
    #这里可以加模型中的方法
    #……
    #保存
    model.save(savePath)

#读模型
def readWord2VecModel(savePath):
    newModel = Word2Vec.load(savePath)
    return newModel

In [25]:
df['text_list'] = df['text'].apply(lambda x:x.split())

In [26]:
dataList = list(df['text_list'].values)

In [ ]:
# trainWord2vec(dataList,"data//dataWord2Vec")


In [20]:
# dataWord2Vec = readWord2VecModel("data//dataWord2Vec")

In [ ]:
# dataWord2Vec.wv.vocab.keys()

In [22]:
# dataWord2Vec['6758']

G:\ph\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.20526437,  0.25692332, -0.19281515, -0.06688049,  0.00972781,
        0.15549842,  0.25335723, -0.13391559, -0.04933024, -0.0709722 ,
        0.10698353,  0.20760068, -0.0171919 ,  0.05800337, -0.07830507,
        0.10510696, -0.32054543, -0.33522087, -0.01314532,  0.01824752,
       -0.07801764, -0.11822597,  0.10061953, -0.01857546,  0.04716896,
       -0.17063451, -0.25374296, -0.12003005, -0.32022393, -0.05361705,
       -0.09556497,  0.11839112, -0.02262084,  0.02047583, -0.06488313,
        0.06642535, -0.0056026 ,  0.5001628 ,  0.0399593 ,  0.17953101,
        0.01899508,  0.26151606,  0.2785155 ,  0.03797152, -0.04676527,
        0.00943195,  0.03291901, -0.16415478,  0.00297387, -0.00206015,
        0.12788042,  0.04247842, -0.01513307, -0.1937139 , -0.24787813,
        0.12838095, -0.01673364,  0.12094463, -0.06302661, -0.19069491,
       -0.11849233, -0.1599827 , -0.06362164,  0.15121706, -0.09405729,
       -0.09155288, -0.09097308,  0.20847571,  0.15890183, -0.14

In [30]:
#TFidf
# 词频逆文档频率-与上面单纯的数数相关，这里带了概率特征
tfidf = TfidfVectorizer(ngram_range = (1,3), max_features=3000)
dataVec = tfidf.fit_transform(df['text'])

In [31]:
# 划分数据集
train_x = dataVec[:15000]
train_y = df['label'][:15000]
test_x = dataVec[15000:]
test_y = df['label'][15000:]
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(15000, 3000)
(15000,)
(7184, 3000)
(7184,)


In [42]:
#textCNN文本分类实现
def CNN_model(train_x,train_y,test_x,test_y):
    model = keras.models.Sequential()
#     model.add(Embedding(len(vocab)+1,300,input_length=50))#传入编码转向量
    model.add(keras.layers.Conv1D(256,5,padding='same'))
    model.add(keras.layers.MaxPooling1D(3,3,padding='same'))
    model.add(keras.layers.Conv1D(128,5,padding='same'))
    model.add(keras.layers.MaxPooling1D(3,3,padding='same'))
    model.add(keras.layers.Conv1D(64,3,padding='same'))
    model.add(keras.layers.Flatten)
    model.add(keras.layers.Dropout(0.1))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(256,activation='relu'))
    model.add(keras.layers.Dropout(0.1))
    model.add(keras.layers.Dense(14,activiation = 'softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.fit(train_x,train_y,epochs=2,batch_size=800)
    y_pred = model.predict_classes(test_x)
    y_pred = list(map(str,y_predict))
    print("准确率",metrics.accuracy_score(test_y,y_pred))
    print("f1，",metrics.f1_score(test_y,y_pred,average='weighted'))

In [ ]:
CNN_model(train_x,train_y,test_x,test_y)#【待调试】

In [48]:
#RNN
def LSTM_model(train_x,train_y,test_x,test_y):
    model = keras.models.Sequential()
#     model.add(Embedding(len(vocab)+1,300,input_length=50))#传入编码转向量
    model.add(keras.layers.LSTM(128,dropout= 0.2))
    model.add(keras.layers.Dense(128))
    model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.Dropout(0.1))
    model.add(keras.layers.Dense(14,activation="softmax"))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.fit(train_x,train_y,epochs=2,batch_size=800,validation_data=(test_x,test_y))
    score,acc  = model.evaluate(test_x,test_y,batch_size=800)
    print("score:",score)
    print("acc",acc)

In [ ]:
LSTM_model(train_x.toarray(),train_y,test_x.toarray(),test_y)#【输入维度待调】